# Algorithme EM

In [1]:
%matplotlib inline
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

C:\Users\srotc_000\Anaconda3\lib\site-packages\matplotlib\__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [110]:
wine = pd.read_csv('Datasets/Debug.csv', delimiter = ";")

In [120]:
d = 11  #descripteurs
N =  wine.shape[0] #produits à tester
R = 5 #experts

y = np.array(wine.ix[:,d+2:]) #labels des annotateurs
x = np.array(wine.ix[:,0:d]) #variables explicatives
w0 = np.random.rand(1,d)

## Initialisation

In [121]:
def init_mu(y):
	mu = []
	for i in range(0,N):
		mu.append(np.sum(y[i])/R)
	return mu

mu0 = init_mu(y)

def sigma(z):
    return 1/(1+np.exp(-z))

## Etape E

In [122]:
#E-step

def ai(alpha,y):
    a = []
    for i in range(0,N):
        proda = 1
        for j in range(0,R):
            proda = proda*alpha[j]**(y[i][j])*(1-alpha[j])**(1-y[i][j])
        a.append(proda)
    return a

def bi(beta,y):
    b = []
    for i in range(0,N):
        prodb = 1
        for j in range(0,R):
            prodb = prodb*beta[j]**(1-y[i][j])*(1-beta[j])**(y[i][j])
        b.append(prodb)
    return b

def pi(x,w):
    p = []
    for i in range(0,N):
        p.append(sigma(x[i].dot(w.T)))
    return p

def mui(a,b,p):
    mu = []
    for i in range(0,N):
        mu.append(a[i]*p[i]/(a[i]*p[i]+b[i]*(1-p[i])))
    return mu

def E_step(x,y,alpha,beta,w):
    CE = 0 #Conditionnal excepectation
    a = ai(alpha,y)
    b = bi(beta,y)
    p = pi(x,w)
    mu = mui(a,b,p)
    for i in range(0,N):
        CE += mu[i]*np.log(p[i])*a[i]+(1-mu[i])*np.log(1-p[i])*b[i]
    return mu


## Maximum Log - Likelihood Estimator

In [123]:
def logLikelihood(w, y, alpha, beta):
    a = ai(alpha, y)
    b = bi(beta, y)
    p = pi(x, w)
    
    #On calcule directement la log-vraissemblance.
    vraissemblance = 0
    for i in range(0,N):
        vraissemblance = vraissemblance + np.log((a[i]*p[i])+b[i]*(1-p[i]))
        
    return vraissemblance

## Étape M

In [124]:
#M-step

def alpha_function(mu,y):
	alpha = []
	for j in range(0,R):
		tmp1 = 0
		tmp2 = 0
		for i in range(0,N):
			tmp1 += mu[i]*y[i][j]
			tmp2 += mu[i]
		alpha.append(tmp1/tmp2)
	return alpha

def beta_function(mu,y):
	beta = []
	for j in range(0,R):
		tmp1 = 0
		tmp2 = 0
		for i in range(0,N):
			tmp1 += (1-mu[i])*(1-y[i][j])
			tmp2 += 1-mu[i]
		beta.append(tmp1/tmp2)
	return beta

def updateW(w,x, eta, mu):
    g = 0
    for i in range(0,N):
        g += (mu[i] - sigma(x[i].dot(w.T)))*x[i]
    
    H = np.zeros((d,d))
    for i in range(0,N):
        H -= sigma(x[i].dot(w.T))*(1-sigma(x[i].dot(w.T)))*((x[i].reshape(11,1))*(x[i].reshape(1,11)))
    w = w - eta*np.linalg.inv(H).dot(g)
    return w

In [125]:
updateW(w0,x,0.1,mu0)

array([[ -5.75612410e+12,   1.35164398e+13,  -3.36551203e+13,
          4.98942796e+12,   5.89299859e+13,  -2.74644506e+12,
         -3.07481100e+12,   2.38470728e+14,  -4.39661631e+13,
          7.92000067e+13,  -6.42045895e+12]])

## Itérations

In [131]:
mu = init_mu(y)
alpha = alpha_function(mu,y)
beta = beta_function(mu,y)
diff_w = 10
w = w0

compteur = 0

#while (np.linalg.norm(diff_w) > 0.001) : # Limite de convergence à decider
while (compteur < 1000):
    mu = E_step(x,y,alpha,beta,w)
    alpha = alpha_function(mu,y)
    beta = beta_function(mu,y)
    w_bis = updateW(w,x,0.01,mu)
    diff_w = w - w_bis
    w = w_bis
    if (compteur % 100 == 0):
        print ("ITERATION : ", compteur)
        print("Vraissemblance : ", logLikelihood(w, y, alpha, beta))
        print("Norme de diff_w : ", np.linalg.norm(diff_w))
        print("Alpha : ", alpha)
        print("Beta : ", beta)
    compteur = compteur + 1

ITERATION :  0
Vraissemblance :  [-364.8077508]
Norme de diff_w :  0.203736988181
Alpha :  [array([ 0.7804878]), array([ 0.62601626]), array([ 0.69105691]), array([ 0.58536585]), array([ 0.18699187])]
Beta :  [array([ 0.05048238]), array([ 0.28537197]), array([ 0.26491983]), array([ 0.57698059]), array([ 0.90220447])]
ITERATION :  100
Vraissemblance :  [-363.22091839]
Norme de diff_w :  13.889383926
Alpha :  [array([ 0.77901023]), array([ 0.62349891]), array([ 0.68897736]), array([ 0.58257488]), array([ 0.18151937])]
Beta :  [array([ 0.]), array([ 0.]), array([ 0.]), array([ 0.]), array([ 0.])]
ITERATION :  200
Vraissemblance :  [-359.15905844]
Norme de diff_w :  1.4802167216
Alpha :  [array([ 0.77695079]), array([ 0.61999023]), array([ 0.68607889]), array([ 0.57868483]), array([ 0.17389181])]
Beta :  [array([ 0.]), array([ 0.]), array([ 0.]), array([ 0.]), array([ 0.])]
ITERATION :  300
Vraissemblance :  [-358.44468312]
Norme de diff_w :  1.11413738656
Alpha :  [array([ 0.7768833]), a